In [1]:
import numpy as np
import pandas as pd
import os, sys

basepath = os.path.expanduser('~/Desktop/src/Amazon_Employee_Access_Challenge/')
sys.path.append(os.path.join(basepath, 'src'))

np.random.seed(2016)

In [2]:
# load files
train   = pd.read_csv(os.path.join(basepath, 'data/raw/train.csv'))
test    = pd.read_csv(os.path.join(basepath, 'data/raw/test.csv'))
sub     = pd.read_csv(os.path.join(basepath, 'data/raw/sampleSubmission.csv'))  

In [3]:
train.shape

(32769, 10)

In [4]:
test.shape

(58921, 10)

** Lets see which values are only seen in the training set and not in test set for every feature. **
** We will do for test set as well. **

In [15]:
def unique_values(df_first, df_second, features):
    """
    Prints out values that are only unique to df_first for each of the features
    """
    
    for feat in features:
        first_feat      = set(df_first[feat].unique())
        second_feat     = set(df_second[feat].unique())
        unique_to_first = first_feat - second_feat
        
        print('Feature %s: \n'%(feat))
        print(unique_to_first)
        print('='*100 + '\n')

In [16]:
unique_values(train, test, train.columns[1:-1])

Feature RESOURCE: 

{40961, 40962, 40963, 73753, 73755, 16430, 106545, 16433, 16434, 16437, 73786, 73787, 16442, 73793, 16449, 106563, 73803, 16461, 106581, 106584, 98394, 16488, 114810, 98430, 98431, 16510, 98433, 16516, 98437, 73861, 73863, 114821, 114822, 114823, 114826, 16528, 73874, 16530, 73879, 82090, 114862, 41136, 98500, 73925, 199, 98505, 73933, 32987, 233, 16619, 73969, 16630, 57598, 73987, 90372, 82181, 33031, 90377, 33035, 73995, 41228, 33038, 33039, 33040, 41230, 41233, 82194, 114963, 74007, 90393, 82202, 82209, 33058, 82211, 33060, 33061, 82212, 33063, 33064, 74033, 74037, 24885, 33079, 8504, 24889, 24890, 24895, 24896, 115010, 115013, 24902, 33095, 41288, 82249, 41303, 41307, 41308, 33117, 90460, 33119, 33120, 90461, 82277, 16742, 74089, 74096, 74099, 106868, 74101, 33142, 74103, 74102, 16760, 90483, 90489, 41340, 41347, 33156, 41348, 16773, 90510, 16783, 82326, 33175, 33176, 41370, 82330, 41373, 90525, 41376, 90529, 41379, 33189, 74152, 33193, 74163, 41397, 74166, 7417

In [17]:
unique_values(test, train, train.columns[1:-1])

Feature RESOURCE: 

set()

Feature MGR_ID: 

{69640, 141330, 20498, 6162, 112680, 24623, 53299, 51255, 4161, 75846, 53323, 90198, 86108, 24672, 8289, 59489, 59495, 6247, 53356, 53357, 39024, 4213, 51329, 132, 51332, 28806, 36998, 22664, 2189, 4239, 2192, 2195, 2197, 20633, 6297, 30874, 37032, 4271, 51380, 4279, 26812, 84160, 71874, 20676, 116934, 75975, 10438, 57548, 39119, 16597, 164056, 49373, 311517, 4325, 65775, 16628, 88312, 8449, 4355, 119045, 78086, 86279, 53511, 10507, 71952, 121104, 121112, 84253, 4383, 90401, 65828, 51493, 28965, 20775, 4391, 49451, 84270, 37169, 14642, 16693, 14646, 28983, 14649, 49468, 22845, 22846, 72000, 59713, 49474, 102721, 4419, 6464, 63815, 4424, 108873, 22857, 332, 57678, 98641, 129364, 115030, 117081, 24924, 31068, 22877, 22881, 55650, 92523, 55659, 8558, 72054, 43385, 14715, 49534, 53631, 92544, 31106, 57735, 47497, 72076, 22933, 16796, 14751, 24996, 76198, 57767, 27048, 20906, 16811, 27055, 20911, 311730, 27059, 41402, 16828, 215488, 4544, 31171, 

** Resource. **

In [19]:
joint_freq_table = pd.crosstab(train.RESOURCE, train.ACTION)

In [22]:
joint_prob = joint_freq_table / joint_freq_table.sum().sum()

In [31]:
def get_prob(resource_id):
    action = 1 # whether resource should be granted or not
    
    numerator = joint_prob.ix[resource_id, action]
    normalized_resource_prob = joint_prob.ix[resource_id].sum()
    
    return numerator / normalized_resource_prob

In [32]:
test.head()

,id,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,78766,72734,118079,118080,117878,117879,118177,19721,117880
1,2,40644,4378,117961,118327,118507,118863,122008,118398,118865
2,3,75443,2395,117961,118300,119488,118172,301534,249618,118175
3,4,43219,19986,117961,118225,118403,120773,136187,118960,120774
4,5,42093,50015,117961,118343,119598,118422,300136,118424,118425


In [46]:
action_preds = list(map(get_prob, test.RESOURCE))

In [47]:
sub['Action'] = action_preds

In [49]:
sub.to_csv(os.path.join(basepath, 'submissions/baseline.csv'), index=False)